# The footprint of a letter

## Setup

In [3]:
import requests

url = "https://competitions.aiolympiad.my/api/maio_2025/maio_2025_letter_footprint"

def post_answer(data: dict):
    response = requests.post(url=url, json=data, headers={"X-API-Key": auth_token})
    if response.status_code == 200:
        return response.json()
    else:
        return f"Failed to submit, status code is {response.status_code}\n{response.text}"

## Introduction

You are provided with this model below.

In [4]:
# You need Pytorch for this!
import torch
import torch.nn as nn
import torch.nn.functional as F

In [5]:
# As long as you are on version 2.x of Pytorch, you'll be fine!
torch.__version__

'2.6.0+cu124'

In [6]:
# Do you know why it is important that this seed is fixed?
# Google or ask an LLM assistant if you don't! :)
torch.manual_seed(42)

In [7]:
class SimpleNet(nn.Module):
    def __init__(self, input_size: int, hidden_size: int, output_size: int):
        super().__init__()

        # Define layers
        self.fc1 = nn.Linear(input_size, hidden_size)
        self.relu = nn.ReLU()
        self.fc2 = nn.Linear(hidden_size, output_size)

        # Specific weight initialization
        self._initialize_weights()

    def _initialize_weights(self):
        # Initialize weights with random values
        nn.init.normal_(self.fc1.weight, mean=0.0, std=0.02)
        nn.init.normal_(self.fc2.weight, mean=0.0, std=0.02)

        # Initialize biases to zero
        nn.init.constant_(self.fc1.bias, 0.0)
        nn.init.constant_(self.fc2.bias, 0.0)

    def forward(self, x):
        x = self.fc1(x)
        x = self.relu(x)
        x = self.fc2(x)
        return x

In [8]:
model = SimpleNet(input_size=26, hidden_size=64, output_size=50).to("cpu")

Notice how the input size is 26. This is entirely by design, such that each element of the input corresponds with each letter of the alphabet.

You are also provided with this list of vectors below.

In [9]:
footprint1 = torch.FloatTensor([
    -0.0293, -0.1092, -0.0290,  0.0885,  0.0623, -0.1106, -0.0229,  0.1987,
    -0.1062,  0.1949, -0.0279,  0.0705, -0.2228, -0.0478, -0.1745,  0.1071,
    -0.0549,  0.1062,  0.1074, -0.0229, -0.1208, -0.0866, -0.1071, -0.0919,
     0.1597,  0.1101,  0.0335,  0.0424, -0.1154,  0.0375,  0.0545, -0.2223,
    -0.0270, -0.0340,  0.0261,  0.1256,  0.0372, -0.0898, -0.1757,  0.1671,
    -0.1561,  0.0908,  0.0166, -0.0705, -0.0078, -0.0120, -0.0082, -0.0245,
    -0.1756, -0.0689
])

In [10]:
footprint2 = torch.FloatTensor([
    -0.1702, -0.2287, -0.0547, 0.0448, 0.2204, 0.0330, -0.1223, 0.0966,
    -0.0118, -0.0911, 0.1477, 0.1065, -0.3109, -0.0817, -0.0824, 0.1230,
    0.1757, 0.1231, -0.0162, -0.1775, -0.1278, -0.1683, 0.1159, 0.1903,
    -0.0053, 0.0484, -0.0476, 0.0044, -0.0514, -0.1619, -0.0399, 0.0002,
    0.0555, -0.2482, -0.0462, -0.4026, -0.0799, -0.3325, -0.0719, 0.2053,
    0.0951, 0.2918, -0.0194, -0.0637, -0.1995, 0.1005, -0.0509, -0.0357,
    -0.2760, -0.1030
])

In [11]:
footprint3 = torch.FloatTensor([
    -0.1158, -0.2736, -0.1917,  0.1027, -0.0901,  0.3210, -0.1885,  0.1472,
     0.1963, -0.1349, -0.2345,  0.5342, -0.2488,  0.2003, -0.0306, -0.1891,
     0.1288,  0.2327,  0.0963, -0.1089, -0.2410,  0.1069, -0.1464, -0.2085,
     0.0420,  0.1267, -0.1581, -0.0614,  0.0604,  0.0312, -0.2559,  0.0039,
     0.3445, -0.0059,  0.0890, -0.0916,  0.1148,  0.0775,  0.0633,  0.1515,
    -0.1137,  0.0521, -0.0418,  0.0388, -0.0424,  0.0375, -0.0586,  0.0932,
    -0.0640, -0.1933
])

In [12]:
footprint4 = torch.FloatTensor([
    0.2047, -0.2309, -0.2069, -0.3843,  0.0616, -0.0781,  0.1256,  0.2333,
    -0.0234, -0.0146, -0.3804,  0.3934, -0.1173, -0.2266, -0.2054,  0.0861,
    -0.0513,  0.1966,  0.1906,  0.0125, -0.3607, -0.2924, -0.0591, -0.3109,
     0.3125,  0.4452, -0.1773, -0.1590, -0.2283, -0.0456, -0.0041, -0.0896,
     0.1555, -0.1307,  0.2646, -0.1352,  0.1714,  0.0815,  0.3392, -0.2495,
     0.2008, -0.0400,  0.0700, -0.1225, -0.3702, -0.2685,  0.0006, -0.2181,
    -0.0386, -0.4234
])

Each vector corresponds to outputs of the network when specific letters are passed through the network. The network takes in letters by accepting a 26-dim vector that is all zeros save for the $i^{th}$ position having a non-zero value, representing the $i^{th}$ letter in the alphabet.

These letters left nothing behind aside from these "footprints".

## Your task

Find the four letters that led to the "footprints" above. When you do, submit your answer following the example below. Make sure your letters are all lowercase and combined into a single string.

```python
post_answer({"solution": "abcd"})
```

This challenge will be graded via both API submission and notebook submission. Scoring as follows:

- 2 pts for submitting the correct string through `post_answer()`. Score will be calculated via autograder. 10 submissions allowed.
- 2 pts if (a) your answer is correct, and (b) your notebook submission demonstrates a method that shows clear logical reasoning with verifiable steps.
- 3 pts if (a) your answer is correct, and (b) your notebook submission shows your method will still have similar runtime even if the number of characters in the alphabet is increased to 10,000.
- 3 pts if (a) your answer is correct, and (b) you can find the numerical values used to represent the letters in the first place! Get within +/-5% for your answers to count.
- Partial credit to be given at discretion

In [13]:
# Your work below :)
footprints = torch.stack([footprint1, footprint2, footprint3, footprint4])

# one-hot encoded letter representations
one_hot_letters = torch.eye(26)  # shape: [26, 26]

# pass all one-hot vectors through the model in a single batch call
with torch.no_grad():
  model_footprints = model(one_hot_letters) # shape: [26, 50]

# compare outputs to provided footprints
# compute cosine similarity between each provided footprint and every model_footprint
similarities = F.cosine_similarity(footprints.unsqueeze(1), model_footprints.unsqueeze(0), dim=2)

# for each provided footprint, pick the letter index with the highest similarity
letter_indices = similarities.argmax(dim=1)

# convert indices to letters and print
alphabet = "abcdefghijklmnopqrstuvwxyz"

identified_letters = ''.join([alphabet[idx] for idx in letter_indices])

print("Identified letters:", identified_letters)

post_answer({"solution": identified_letters})

# compare outputs and compute percent error
for i, idx in enumerate(letter_indices):
  model_output = model_footprints[idx]
  provided = footprints[i]

  # calculate percentage error (avoid division by zero)
  percent_error = (torch.abs(model_output - provided) / (torch.abs(provided) + 1e-8)) * 100

  # display results
  print(f"\nFootprint {i+1} vs. letter '{alphabet[idx]}':")
  print("Model output (first 5 values):", model_output[:5].tolist())
  print("Footprint (first 5 values):", provided[:5].tolist())
  print("Percent error (first 5 values):", percent_error[:5].tolist())


Identified letters: oiam

Footprint 1 vs. letter 'o':
Model output (first 5 values): [-0.0007317269337363541, -0.002731061540544033, -0.0007260831189341843, 0.0022127944976091385, 0.0015582827618345618]
Footprint (first 5 values): [-0.02930000051856041, -0.10920000076293945, -0.028999999165534973, 0.0885000005364418, 0.062300000339746475]
Percent error (first 5 values): [97.50260925292969, 97.49901580810547, 97.4962387084961, 97.49966430664062, 97.49872589111328]

Footprint 2 vs. letter 'i':
Model output (first 5 values): [-0.002837108913809061, -0.0038117575459182262, -0.0009116705041378736, 0.0007459884509444237, 0.0036729788407683372]
Footprint (first 5 values): [-0.17020000517368317, -0.22869999706745148, -0.05469999834895134, 0.04479999840259552, 0.22040000557899475]
Percent error (first 5 values): [98.33306884765625, 98.33329010009766, 98.33330535888672, 98.3348159790039, 98.33348846435547]

Footprint 3 vs. letter 'a':
Model output (first 5 values): [-0.001446984359063208, -0.003